In [26]:
import sys
sys.path.append('/home/silja/fatepredict/fatepredict_experiments')

In [29]:
import zarr
import numpy as np
import networkx as nx
from skimage.measure import regionprops
from segmentation import segment_stats
from linajea_cost_test import get_merge_graph_from_array

In [38]:
z = zarr.open('anno_alice_T2030_tiny.zarr','r')
gt_image = z['gt_trackimage'][:]
raw = z['Raw'][:]
frg = z['Fragments']

In [16]:
# annation label is 10000, 10001, 10004, 10005, 10007, 10010, 10013, 10016, 10017 pick them out
gt_mask = gt_image >=10000
gt_image = gt_mask * gt_image

In [23]:
gt_image.shape

(3, 15, 30, 30)

In [28]:
# get the id list of one time frame
id, positions, _ = segment_stats(gt_image[0],0)

In [32]:
t = 0
merge_tree = z['Merge_tree/Merge/'+str(t)]
scores = z['Merge_tree/Scoring/'+str(t)]
merge_tree = get_merge_graph_from_array(merge_tree,scores)


In [51]:

def count_anno_id(id,merge_tree):
    '''
    merge_tree: nx.Digrph merge_tree create by waterz 
    id: ndarray (n,) 1d array store all ids in GT image
    correct_id_num: int count the number of how many annotation id in merge tree
    '''
    mt_id = list(merge_tree.nodes)
    correct_id_num = len(set(id)&set(mt_id))
    return correct_id_num 

correct_id_num = count_anno_id(id,merge_tree)

In [37]:
import napari
viewer = napari.Viewer()
viewer.add_image(raw[3])
viewer.add_labels(gt_image)

<Labels layer 'gt_image' at 0x7fa0a65407f0>

# get the sovler output

In [52]:
# read .gexf file which created by run_candidate_graph.ipynb
graph = nx.read_gexf('text.gexf')

In [53]:
def pick_select_edge(graph):
    edges = []
    selected_edges = []
    for u,v,data in graph.edges(data=True):
        edges.append((u,v))
        if data['selected']:
            selected_edges.append((u,v))
    return edges, selected_edges

edges, selected_edges = pick_select_edge(graph)

In [58]:
graph.nodes(data=True)


NodeDataView({'19140315621363712': {'t': 0, 'z': 8, 'y': 12, 'x': 2, 'score': 0.04924517869949341, 'parent': 53339542484161024, 'label': '19140315621363712'}, '8584995398486528': {'t': 0, 'z': 5, 'y': 9, 'x': 1, 'score': 0.0001, 'parent': 19140315621363712, 'label': '8584995398486528'}, '10555337427916288': {'t': 0, 'z': 11, 'y': 15, 'x': 3, 'score': 0.0001, 'parent': 19140315621363712, 'label': '10555337427916288'}, '53339542484161024': {'t': 0, 'z': 9, 'y': 18, 'x': 4, 'score': 0.08333110809326172, 'parent': 54465442391003648, 'label': '53339542484161024'}, '34199252666160128': {'t': 0, 'z': 10, 'y': 22, 'x': 5, 'score': 0.0001, 'parent': 53339542484161024, 'label': '34199252666160128'}, '15129503355377664': {'t': 0, 'z': 12, 'y': 9, 'x': 26, 'score': 0.11383092403411865, 'parent': 17944253120387072, 'label': '15129503355377664'}, '12420306700342784': {'t': 0, 'z': 11, 'y': 7, 'x': 26, 'score': 0.0001, 'parent': 15129503355377664, 'label': '12420306700342784'}, '2709428612635136': {'